## Reading Values from Touch Keypad

This demonstration shows how to interact with the Raspberry Pi Touch Keypad.
The Raspberry Pi Touch Keypad is required (https://www.seeedstudio.com/Raspberry-Pi-Touch-Keypad-p-2772.html).

![](data/rpi_touchpad.jpg)

The Raspberry Pi touch keypad supports up to 16 keys with adjustable 
sensitivity and built-in LD0.
Touch keypad is read only, and has IIC interface
connected to SDA1 and SCL1 on the Raspberry Pi interface.
The I2C will read 2 bytes of data: `Data_0` and `Data_1`.
* `Data_0`: B7 ~ B0 is TP0 ~ TP7 on/off status. 0 is key off, 1 is key on.
* `Data_1`: B7 ~ B0 is TP8 ~ TP15 on/off status. 0 is key off, 1 is key on.

### 1. Prepare the overlay
Download the overlay first, then select the shared pin to be connected to
RPI header (by default, the pins will be connected to PMODA instead).

In [1]:
from pynq.overlays.base import BaseOverlay

base = BaseOverlay("base.bit")
base.select_rpi()

### 2. Instantiate the Microblaze
The Microblaze will control the pins on the RASPBERRYPI header.

In [2]:
%%microblaze base.RPI

#include "xio_switch.h"
#include "circular_buffer.h"
#include "i2c.h"

// Device constants
#define TOUCHPAD_DEV_ADDRESS         0x57

unsigned int get_touchpad_reg_value(){
    uint8_t data[2];
    i2c device = i2c_open_device(1);
    set_pin(2, SDA1);
    set_pin(3, SCL1);
    i2c_read(device, TOUCHPAD_DEV_ADDRESS, data, 2);
    return (unsigned int) ((data[0] << 8) + data[1]);
}

### 3. Read the key values
The available pin names are listed below.

In [3]:
PIN_MAPPING = {'circle': 0,
               'cross': 1,
               'square': 2,
               'r': 3,
               'home': 4,
               '+': 5,
               '-': 6,
               'l': 7,
               'down': 8,
               'right': 9,
               'up': 10,
               'left': 11,
               'power': 12,
               'rpi': 13,
               'logo': 14,
               'triangle': 15
               }

To convert the raw data into the value for each key, we define the following
functions.

In [4]:
def reg2int(reg_value, key_number):
    return "{0:b}".format(reg_value).zfill(16)[-(key_number+1)]

def get_touchpad(key_name):
    reg_value = get_touchpad_reg_value()
    key_number = PIN_MAPPING[key_name]
    return reg2int(reg_value, key_number)

Run the following code without pressing any button.

In [5]:
get_touchpad('home')

'0'

While pressing gently on the home button of the touch keypad, run the following code.

In [6]:
get_touchpad('home')

'1'

While pressing the right arrow and square at the same time, 
run the following code. Note that there are 2 read commands issued,
although 1 read command can get values for all the keys.

In [7]:
for key in ['right', 'square']:
    print('Key {} reads value {}.'.format(key, get_touchpad(key)))

Key right reads value 1.
Key square reads value 1.


### 4. Cleanup
Switch back the connection on the shared pin to PMODA header.

In [8]:
base.select_pmoda()